## Machine Learning TP1 : Reconnaissance de Champignon

### Import des librairies

In [ ]:
import pandas as pd
import numpy as np
import os
from IPython.display import Image, display

from sklearn import preprocessing                       # préparation des données

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import ensemble
from sklearn.tree import export_graphviz
# pd.set_option('display.max_rows', None) # Afficher toutes les lignes des dataframes
# pd.set_option('display.max_columns', None) # Afficher toutes les lignes des dataframes

### 1. Preparation des données

1.1 importer et afficher les données dans un DataFrame.

Afficher le nombre de champignons toxiques ou non.

Vérifier qu'il n'y as pas de valeurs nulles

In [2]:
df_mushrooms = pd.read_csv('data/champignons.csv')
df_mushrooms.head()

,toxicite,surface-chapeau,ecchymoses,attache-lamelles,taille-lamelles,forme-pied,surface-pied-sur-anneau,couleur-pied-sur-anneau,couleur-voile,type-anneau,population
0,toxique,lisse,oui,libre,étroit,élargissant,lisse,blanc,blanc,pendant,dispersée
1,comestible,lisse,oui,libre,large,élargissant,lisse,blanc,blanc,pendant,nombreuse
2,comestible,lisse,oui,libre,large,élargissant,lisse,blanc,blanc,pendant,nombreuse
3,toxique,écaillée,oui,libre,étroit,élargissant,lisse,blanc,blanc,pendant,dispersée
4,comestible,lisse,non,libre,large,effilé,lisse,blanc,blanc,évasé,abondante


In [3]:
print("\nValeurs manquantes par colonne :")
print(df_mushrooms.isnull().sum())


Valeurs manquantes par colonne :
toxicite                   0
surface-chapeau            0
ecchymoses                 0
attache-lamelles           0
taille-lamelles            0
forme-pied                 0
surface-pied-sur-anneau    0
couleur-pied-sur-anneau    0
couleur-voile              0
type-anneau                0
population                 0
dtype: int64


In [4]:
print("Nombre de champignons toxiques ou non")
print(df_mushrooms['toxicite'].value_counts())
# Vérifier les valeurs nulles dans la colonne toxicité
print(f"Nombre de valeurs nulles dans 'toxicité' : {df_mushrooms['toxicite'].isnull().sum()}")

Nombre de champignons toxiques ou non
toxicite
comestible    4208
toxique       3916
Name: count, dtype: int64
Nombre de valeurs nulles dans 'toxicité' : 0


1.2 Encoder les données avec une colonne 1/0 par valeur possible.Eviter de générer une matrice "sparse"\
Afficher la taille des données (lignes x colonnes) et les noms de colonnes générées.\
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder

In [10]:
# Séparation des variables (comestible / toxique)
toxicity = df_mushrooms['toxicite']                                 # variable cible: toxicité
mushrooms_features = df_mushrooms.drop('toxicite', axis=1)          # feature avant encoding

# Encoder avec OneHotEncoder
encoder = preprocessing.OneHotEncoder(sparse_output=False, dtype=int)
encoder.fit(mushrooms_features)
mushrooms_encoded = encoder.transform(mushrooms_features)  # feature de données transformées en numpy array

# Afficher la taille des données
print(f"Taille de df_mushrooms original: {df_mushrooms.shape}")
print(f"Taille de mushrooms_features : {mushrooms_features.shape}")
print(f"Taille de mushrooms_encoded : {mushrooms_encoded.shape}")

print("\n")

# Récupérer les noms des colonnes générées
feature_names = encoder.get_feature_names_out(mushrooms_features.columns)

# Affichage en regroupant par feature d'origine
for col in mushrooms_features.columns:
    # Filtrer les colonnes qui correspondent à cette feature
    cols_for_feature = [name for name in feature_names if name.startswith(col)]
    print(f"\n{col} ({len(cols_for_feature)} modalités) :")
    for col_name in cols_for_feature:
        print(f"  - {col_name}")


Taille de df_mushrooms original: (8124, 11)
Taille de mushrooms_features : (8124, 10)
Taille de mushrooms_encoded : (8124, 40)



surface-chapeau (4 modalités) :
  - surface-chapeau_fibreuse
  - surface-chapeau_lisse
  - surface-chapeau_rainurée
  - surface-chapeau_écaillée

ecchymoses (2 modalités) :
  - ecchymoses_non
  - ecchymoses_oui

attache-lamelles (2 modalités) :
  - attache-lamelles_adhérent
  - attache-lamelles_libre

taille-lamelles (2 modalités) :
  - taille-lamelles_large
  - taille-lamelles_étroit

forme-pied (2 modalités) :
  - forme-pied_effilé
  - forme-pied_élargissant

surface-pied-sur-anneau (4 modalités) :
  - surface-pied-sur-anneau_fibreuse
  - surface-pied-sur-anneau_lisse
  - surface-pied-sur-anneau_soyeuse
  - surface-pied-sur-anneau_écaillée

couleur-pied-sur-anneau (9 modalités) :
  - couleur-pied-sur-anneau_beige
  - couleur-pied-sur-anneau_blanc
  - couleur-pied-sur-anneau_brun
  - couleur-pied-sur-anneau_cannelle
  - couleur-pied-sur-anneau_gris
  - coul

1.3 Créer un nouveau DataFrame avec les données et les noms de colonne (méthode get_feature_names_out() de OneHotEncoder). Mélanger les données. Afficher le résultat.

In [16]:
# DataFrame avec les données encodées
df_mushrooms_encoded = pd.DataFrame(mushrooms_encoded,columns=feature_names,dtype=int)

# DataFrame avec données mélangées
df_mushrooms_encoded_shuffled = df_mushrooms_encoded.sample(frac=1, random_state=42).reset_index(drop=True)

print("\nInformations sur le DataFrame encodé :")
df_mushrooms_encoded_shuffled.info()

print("\n")

print("\nPremières lignes (après mélange) :")
df_mushrooms_encoded_shuffled.head()



Informations sur le DataFrame encodé :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 40 columns):
 #   Column                            Non-Null Count  Dtype
---  ------                            --------------  -----
 0   surface-chapeau_fibreuse          8124 non-null   int64
 1   surface-chapeau_lisse             8124 non-null   int64
 2   surface-chapeau_rainurée          8124 non-null   int64
 3   surface-chapeau_écaillée          8124 non-null   int64
 4   ecchymoses_non                    8124 non-null   int64
 5   ecchymoses_oui                    8124 non-null   int64
 6   attache-lamelles_adhérent         8124 non-null   int64
 7   attache-lamelles_libre            8124 non-null   int64
 8   taille-lamelles_large             8124 non-null   int64
 9   taille-lamelles_étroit            8124 non-null   int64
 10  forme-pied_effilé                 8124 non-null   int64
 11  forme-pied_élargissant            8124 non-null   int64

,surface-chapeau_fibreuse,surface-chapeau_lisse,surface-chapeau_rainurée,surface-chapeau_écaillée,ecchymoses_non,ecchymoses_oui,attache-lamelles_adhérent,attache-lamelles_libre,taille-lamelles_large,taille-lamelles_étroit,...,type-anneau_fibreux,type-anneau_large,type-anneau_pendant,type-anneau_évasé,population_abondante,population_clairsemée,population_dispersée,population_nombreuse,population_solitaire,population_variée
0,1,0,0,0,1,0,0,1,1,0,...,0,0,0,1,0,0,1,0,0,0
1,0,1,0,0,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,1
2,0,0,0,1,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,1
3,0,0,0,1,0,1,0,1,1,0,...,0,0,1,0,0,0,0,0,1,0
4,0,1,0,0,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,1


### On a vérifié le One Hot Encoding

1.4 Creer 4 jeux de données : train features et labels (70%) et dev features et labels (30%).\
Utiliser toxicite_toxique comme label. Ne pas garder de label (toxicite_toxique et toxicite_comestible) dans les features !

## 2. Arbre de décision

2.1 Entrainer un arbre de décision sur les données d'apprentissage. Afficher l'arbre obtenu en utilisant plot_tree.

In [ ]:
import matplotlib.pyplot as plt

2.2 Afficher l'arbre obtenu en utilisant graphwiz

2.3 Mesurer la précision du modèle sur les données de dev

## 3. Modèle de forêt

3.1 Réalisez un algorithme de Random Forest sur les données. Utiliser 500 arbres et une profondeur de 6.

3.2 Afficher trois arbres au hasard.

In [ ]:
from random import randint


3.3 Mesurer la précision du modèle Random Forest sur les données de dev

## 4. optimisation des paramètres

4.1 En utilisant GridSearchCV, chercher la meilleure combinaison max_depth et n_estimator.

Afficher les paramètre du modèle et a précision obtenue.

Réexécuter un apprentissage avec la meilleure combinaison.


In [ ]:
from sklearn.model_selection import GridSearchCV


4.2 Afficher les paramètres optimaux et la précisions obtenue sur les données dev

4.3 Classer et Afficher les caractéristiques d'importance supérieure à 1%

## 5. Stacking

Essayer différents modèles d'estimateur final

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.ensemble import StackingClassifier


## 6. Gradient Boosting

6. Exécuter un apprentissage de Gradient Boosting avec et sans "Early Stopping" sur le nombre d'estimateurs. Comparer avec le résultat sur les données dev.

## 7. (Optionnel) Recommencer les exercices précédents en utilisant les "ordinal encoders". Que remarquez-vous ?